#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [10]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [11]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [13]:
embeddings=OllamaEmbeddings(model="gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

In [17]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs


[Document(id='d03bee3e-3898-4e39-a5ad-2489ea59d0e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b2d66520-2639-4f5f-acfc-8958bfdcf905', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a pat

In [18]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [20]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs

[Document(id='d03bee3e-3898-4e39-a5ad-2489ea59d0e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b2d66520-2639-4f5f-acfc-8958bfdcf905', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a pat

In [21]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [22]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='d03bee3e-3898-4e39-a5ad-2489ea59d0e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(0.5565837)),
 (Document(id='b2d66520-2639-4f5f-acfc-8958bfdcf905', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that

In [23]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.006574645,
 -0.02792638,
 -2.973353e-05,
 0.01910844,
 0.008948616,
 0.0071993344,
 -0.009341916,
 -0.0017564726,
 -0.0032380102,
 -0.008390064,
 0.017373657,
 -0.00039809613,
 -0.0140977865,
 0.010326056,
 0.003632351,
 -0.0107774595,
 0.0436577,
 0.021817928,
 0.017134516,
 0.012942879,
 0.0034886566,
 -0.005934182,
 0.003322234,
 0.017727436,
 -0.0007951715,
 -0.008113068,
 -0.017155401,
 -0.019503552,
 -0.00607368,
 -0.022429066,
 -0.0055351015,
 -0.01589016,
 0.018473223,
 -0.013295339,
 -0.0029868113,
 -0.005354434,
 0.022940638,
 0.007443943,
 0.0009103502,
 -0.004406921,
 0.0041045365,
 0.012739065,
 0.013913783,
 -0.017139424,
 -0.01472163,
 0.004857908,
 0.0027746744,
 0.016482724,
 -0.009603311,
 0.01571842,
 -0.19068179,
 -0.23054112,
 -0.0064846515,
 0.0046968185,
 0.012799103,
 -0.004160556,
 -0.019582288,
 0.008569371,
 0.0061025494,
 -0.0056186933,
 0.003949382,
 -0.016224064,
 0.00068669085,
 -0.015107691,
 -0.007368603,
 0.0016592435,
 -0.008860086,
 -0.0065725204,

In [24]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='d03bee3e-3898-4e39-a5ad-2489ea59d0e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b2d66520-2639-4f5f-acfc-8958bfdcf905', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a pat

In [25]:
### Saving And Loading
db.save_local("faiss_index")

In [26]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [27]:
docs

[Document(id='d03bee3e-3898-4e39-a5ad-2489ea59d0e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b2d66520-2639-4f5f-acfc-8958bfdcf905', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a pat